In [8]:
import yfinance as yf
import numpy as np
import cvxpy as cp
import scipy.linalg

# 1. 투자기간과 티커 설정
start_date = '2018-01-01'
end_date = '2024-12-31'

tickers = ['SPY', 'QQQ', 'GLD', 'SCHD', 'TLT']

# 2. 데이터 다운로드 및 수익률, 공분산 계산 (일별)
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# 'Adj Close' 컬럼 확인 후 존재하면 사용, 없으면 'Close' 사용
if ('Adj Close' in data.columns.get_level_values(1)):
    adj_close = data.loc[:, (slice(None), 'Adj Close')]
else:
    adj_close = data.loc[:, (slice(None), 'Close')]

adj_close.columns = adj_close.columns.droplevel(1)  # 다중 인덱스 제거
adj_close = adj_close.dropna()

returns = adj_close.pct_change().dropna()
mu = returns.mean() * 252
Sigma = returns.cov() * 252

# 연환산 수익률, 공분산
mu = returns.mean() * 252
Sigma = returns.cov() * 252

# 무위험 이자율 (예: 1.7%)
rf = 0.017

# 3. 최적화 함수 정의
def optimize_portfolio(base_weights, maximize="sharpe", risk_limit=None):
    base = np.array([base_weights.get(t, 0) for t in tickers])
    w = cp.Variable(len(tickers))
    expected_return = mu.values @ w
    
    L = scipy.linalg.cholesky(Sigma.values, lower=True)
    portfolio_vol = cp.norm(L.T @ w)
    
    if maximize == "sharpe":
        # Sharpe ratio 직접 최대화는 DCP 규칙 위반 → 변형 필요
        # 대신 아래처럼 근사하거나
        # 간단히 objective를 수익률 - rf - alpha * 위험으로 변경 가능
        alpha = 1.0  # 조정 가능
        objective = cp.Maximize(expected_return - rf - alpha * portfolio_vol)
    elif maximize == "return":
        objective = cp.Maximize(expected_return)
    else:
        raise ValueError("maximize 파라미터는 'sharpe' 혹은 'return' 이어야 합니다.")
    
    constraints = [
        cp.sum(w) == 1,
        w >= 0,
    ]
    
    if risk_limit is not None:
        constraints.append(portfolio_vol <= risk_limit)
    
    problem = cp.Problem(objective, constraints)
    problem.solve(verbose=False)
    
    weights = w.value.round(4)
    sharpe = (expected_return.value - rf) / portfolio_vol.value
    
    result = {
        'weights': dict(zip(tickers, weights)),
        'expected_return': expected_return.value,
        'volatility': portfolio_vol.value,
        'sharpe': sharpe
    }
    return result

# 4. 기본 포트폴리오 후보들
base_portfolios = {
    'aggressive_3': {'SPY': 0.2507, 'QQQ': 0.2987, 'GLD': 0.2235, 'SCHD': 0.2271, 'TLT': 0.0},
    'aggressive_11': {'SPY': 0.25, 'QQQ': 0.53, 'GLD': 0.22, 'SCHD': 0.0, 'TLT': 0.0},
    'aggressive_9': {'SPY': 0.7, 'QQQ': 0.0, 'GLD': 0.3, 'SCHD': 0.0, 'TLT': 0.0},
    'defensive_23': {'SPY': 0.2501, 'QQQ': 0.0, 'GLD': 0.2485, 'SCHD': 0.2847, 'TLT': 0.2167},
    'defensive_1': {'SPY': 0.3926, 'QQQ': 0.0, 'GLD': 0.164, 'SCHD': 0.258, 'TLT': 0.1854}
}

# 5. 최적화 실행 및 결과 출력
for name, base in base_portfolios.items():
    print(f"\n{name} 최적화 결과:")
    res = optimize_portfolio(base, maximize="sharpe")
    for t, w in res['weights'].items():
        print(f"{t}: {w:.4f}")
    print(f"연 기대수익률: {res['expected_return']:.3f}")
    print(f"연 변동성: {res['volatility']:.3f}")
    print(f"Sharpe Ratio: {res['sharpe']:.3f}")

[*********************100%***********************]  5 of 5 completed


aggressive_3 최적화 결과:
SPY: 0.0000
QQQ: 0.4330
GLD: 0.5670
SCHD: 0.0000
TLT: 0.0000
연 기대수익률: 0.148
연 변동성: 0.140
Sharpe Ratio: 0.935

aggressive_11 최적화 결과:
SPY: 0.0000
QQQ: 0.4330
GLD: 0.5670
SCHD: 0.0000
TLT: 0.0000
연 기대수익률: 0.148
연 변동성: 0.140
Sharpe Ratio: 0.935

aggressive_9 최적화 결과:
SPY: 0.0000
QQQ: 0.4330
GLD: 0.5670
SCHD: 0.0000
TLT: 0.0000
연 기대수익률: 0.148
연 변동성: 0.140
Sharpe Ratio: 0.935

defensive_23 최적화 결과:
SPY: 0.0000
QQQ: 0.4330
GLD: 0.5670
SCHD: 0.0000
TLT: 0.0000
연 기대수익률: 0.148
연 변동성: 0.140
Sharpe Ratio: 0.935

defensive_1 최적화 결과:
SPY: 0.0000
QQQ: 0.4330
GLD: 0.5670
SCHD: 0.0000
TLT: 0.0000
연 기대수익률: 0.148
연 변동성: 0.140
Sharpe Ratio: 0.935


In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

# 1. 설정: 종목, 기간
tickers = ['SPY', 'QQQ', 'GLD', 'SCHD', 'TLT']
start_date = '2018-01-01'
end_date = '2024-12-31'

# 2. 데이터 다운로드 (월봉 종가)
data = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close'].dropna()

# 3. 수익률 계산 (월간 수익률)
returns = data.pct_change().dropna()

# 4. 연환산 수익률 및 공분산 계산
# 월별 평균 수익률 * 12 (연환산), 공분산 행렬도 월별 데이터 기준 연환산
mu = returns.mean() * 12
S = returns.cov() * 12

# 5. base_weights 예시 (공격형 11번)
base_aggressive = {'SPY': 0.25, 'QQQ': 0.53, 'GLD': 0.22, 'SCHD': 0.0, 'TLT': 0.0}

def optimize_portfolio(base_weights, maximize="return", risk_limit=None):
    tickers_order = tickers
    w = cp.Variable(len(tickers_order))

    expected_ret = mu.values @ w
    portfolio_vol = cp.sqrt(cp.quad_form(w, S.values))

    if maximize == "return":
        objective = cp.Maximize(expected_ret)
    elif maximize == "sharpe":
        risk_free = 0.017  # 무위험 수익률
        objective = cp.Maximize((expected_ret - risk_free) / portfolio_vol)

    constraints = [cp.sum(w) == 1, w >= 0]

    # base_weights ± 0.1 범위 내 제한
    for i, t in enumerate(tickers_order):
        base = base_weights.get(t, 0)
        constraints += [w[i] >= max(0, base - 0.1), w[i] <= min(1, base + 0.1)]

    if risk_limit:
        constraints += [portfolio_vol <= risk_limit]

    problem = cp.Problem(objective, constraints)
    problem.solve()

    weights = np.round(w.value, 4)
    result = {
        'weights': dict(zip(tickers_order, weights)),
        'expected_return': expected_ret.value,
        'volatility': portfolio_vol.value,
        'sharpe': ((expected_ret.value - 0.017) / portfolio_vol.value)
    }
    return result

# 6. 최적화 실행 (예: 공격형 11번)
result = optimize_portfolio(base_aggressive, maximize="return")

print("최적화 결과:")
for k, v in result['weights'].items():
    print(f"{k}: {v:.4f}")
print(f"연 기대수익률: {result['expected_return']:.3f}")
print(f"연 변동성: {result['volatility']:.3f}")
print(f"Sharpe Ratio: {result['sharpe']:.3f}")

[*********************100%***********************]  5 of 5 completed

최적화 결과:
SPY: 0.1500
QQQ: 0.6300
GLD: 0.1200
SCHD: 0.1000
TLT: 0.0000
연 기대수익률: 0.159
연 변동성: 0.166
Sharpe Ratio: 0.856


In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

# 1. 설정: 종목, 기간
tickers = ['SPY', 'QQQ', 'GLD', 'SCHD', 'TLT']
start_date = '2018-01-01'
end_date = '2024-12-31'

# 2. 데이터 다운로드 (월봉 종가)
data = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close'].dropna()

# 3. 수익률 계산 (월간 수익률)
returns = data.pct_change().dropna()

# 4. 연환산 수익률 및 공분산 계산
# 월별 평균 수익률 * 12 (연환산), 공분산 행렬도 월별 데이터 기준 연환산
mu = returns.mean() * 12
S = returns.cov() * 12

# 안정형 후보 23번 base_weights 설정
base_defensive = {
    'SPY': 0.2501,
    'QQQ': 0.0,
    'GLD': 0.2485,
    'SCHD': 0.2847,
    'TLT': 0.2167
}

def optimize_portfolio(base_weights, maximize="return", risk_limit=None):
    tickers_order = tickers
    w = cp.Variable(len(tickers_order))

    expected_ret = mu.values @ w
    portfolio_vol = cp.sqrt(cp.quad_form(w, S.values))

    if maximize == "return":
        objective = cp.Maximize(expected_ret)
    elif maximize == "sharpe":
        risk_free = 0.017  # 무위험 수익률
        objective = cp.Maximize((expected_ret - risk_free) / portfolio_vol)

    constraints = [cp.sum(w) == 1, w >= 0]

    # base_weights ± 0.1 범위 내 제한
    for i, t in enumerate(tickers_order):
        base = base_weights.get(t, 0)
        constraints += [w[i] >= max(0, base - 0.1), w[i] <= min(1, base + 0.1)]

    if risk_limit:
        constraints += [portfolio_vol <= risk_limit]

    problem = cp.Problem(objective, constraints)
    problem.solve()

    weights = np.round(w.value, 4)
    result = {
        'weights': dict(zip(tickers_order, weights)),
        'expected_return': expected_ret.value,
        'volatility': portfolio_vol.value,
        'sharpe': ((expected_ret.value - 0.017) / portfolio_vol.value)
    }
    return result

# 6. 최적화 실행
result = optimize_portfolio(base_defensive, maximize="return")

print("최적화 결과:")
for k, v in result['weights'].items():
    print(f"{k}: {v:.4f}")
print(f"연 기대수익률: {result['expected_return']:.3f}")
print(f"연 변동성: {result['volatility']:.3f}")
print(f"Sharpe Ratio: {result['sharpe']:.3f}")

[*********************100%***********************]  5 of 5 completed

최적화 결과:
SPY: 0.2501
QQQ: 0.1000
GLD: 0.1485
SCHD: 0.3847
TLT: 0.1167
연 기대수익률: 0.110
연 변동성: 0.127
Sharpe Ratio: 0.732


In [18]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

# 1. 설정: 종목, 기간
tickers = ['SPY', 'QQQ', 'GLD', 'SCHD', 'TLT']
start_date = '2018-01-01'
end_date = '2024-12-31'

# 2. 데이터 다운로드 (월봉 종가)
data = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close'].dropna()

# 3. 수익률 계산 (월간 수익률)
returns = data.pct_change().dropna()

# 4. 연환산 수익률 및 공분산 계산
mu = returns.mean() * 12
S = returns.cov() * 12

# 5. base_weights 설정 (안정형 후보)
base_defensive = {
    'SPY': 0.2501,
    'QQQ': 0.0,
    'GLD': 0.2485,
    'SCHD': 0.2847,
    'TLT': 0.2167
}

def optimize_portfolio_min_vol(base_weights, min_return=None):
    tickers_order = tickers
    w = cp.Variable(len(tickers_order))

    expected_ret = mu.values @ w
    portfolio_vol = cp.sqrt(cp.quad_form(w, S.values))

    # 목표: 변동성 최소화
    objective = cp.Minimize(portfolio_vol)

    # 제약조건: 합 1, 음수 금지
    constraints = [cp.sum(w) == 1, w >= 0]

    # base_weights ± 0.1 범위 제한 (필요시 조정 가능)
    for i, t in enumerate(tickers_order):
        base = base_weights.get(t, 0)
        constraints += [w[i] >= max(0, base - 0.1), w[i] <= min(1, base + 0.1)]

    # 최소 기대수익률 조건
    if min_return:
        constraints += [expected_ret >= min_return]

    problem = cp.Problem(objective, constraints)
    problem.solve(qcp=True)  # 여기 qcp=True 넣음

    weights = np.round(w.value, 4)
    result = {
        'weights': dict(zip(tickers_order, weights)),
        'expected_return': expected_ret.value,
        'volatility': portfolio_vol.value,
        'sharpe': ((expected_ret.value - 0.017) / portfolio_vol.value)
    }
    return result

# 6. 최적화 실행: 연 기대수익률 5% 이상, 변동성 최소화
result = optimize_portfolio_min_vol(base_defensive, min_return=0.05)

print("최적화 결과 (변동성 최소화):")
for k, v in result['weights'].items():
    print(f"{k}: {v:.4f}")
print(f"연 기대수익률: {result['expected_return']:.3f}")
print(f"연 변동성: {result['volatility']:.3f}")
print(f"Sharpe Ratio: {result['sharpe']:.3f}")

[*********************100%***********************]  5 of 5 completed
D:\Project\Python_Source\Test01\.venv\Lib\site-packages\cvxpy\problems\problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


최적화 결과 (변동성 최소화):
SPY: 0.3501
QQQ: 0.0000
GLD: 0.1486
SCHD: 0.1847
TLT: 0.3166
연 기대수익률: 0.071
연 변동성: 0.106
Sharpe Ratio: 0.509


In [15]:
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp

# 1. 설정: 종목, 기간
tickers = ['SPY', 'QQQ', 'GLD', 'SCHD', 'TLT']
start_date = '2018-01-01'
end_date = '2024-12-31'

# 2. 데이터 다운로드 (월봉 종가)
data = yf.download(tickers, start=start_date, end=end_date, interval='1mo')['Close'].dropna()

# 3. 수익률 계산 (월간 수익률)
returns = data.pct_change().dropna()

# 4. 연환산 수익률 및 공분산 계산
# 월별 평균 수익률 * 12 (연환산), 공분산 행렬도 월별 데이터 기준 연환산
mu = returns.mean() * 12
S = returns.cov() * 12

# 안정형 후보 1번 base_weights 설정
base_defensive = {
    'SPY': 0.3926,
    'QQQ': 0.0,
    'GLD': 0.164,
    'SCHD': 0.258,
    'TLT': 0.1854
}

def optimize_portfolio(base_weights, maximize="return", risk_limit=None):
    tickers_order = tickers
    w = cp.Variable(len(tickers_order))

    expected_ret = mu.values @ w
    portfolio_vol = cp.sqrt(cp.quad_form(w, S.values))

    if maximize == "return":
        objective = cp.Maximize(expected_ret)
    elif maximize == "sharpe":
        risk_free = 0.017  # 무위험 수익률
        objective = cp.Maximize((expected_ret - risk_free) / portfolio_vol)

    constraints = [cp.sum(w) == 1, w >= 0]

    # base_weights ± 0.1 범위 내 제한
    for i, t in enumerate(tickers_order):
        base = base_weights.get(t, 0)
        constraints += [w[i] >= max(0, base - 0.1), w[i] <= min(1, base + 0.1)]

    if risk_limit:
        constraints += [portfolio_vol <= risk_limit]

    problem = cp.Problem(objective, constraints)
    problem.solve()

    weights = np.round(w.value, 4)
    result = {
        'weights': dict(zip(tickers_order, weights)),
        'expected_return': expected_ret.value,
        'volatility': portfolio_vol.value,
        'sharpe': ((expected_ret.value - 0.017) / portfolio_vol.value)
    }
    return result

# 6. 최적화 실행
result = optimize_portfolio(base_defensive, maximize="return")

print("최적화 결과:")
for k, v in result['weights'].items():
    print(f"{k}: {v:.4f}")
print(f"연 기대수익률: {result['expected_return']:.3f}")
print(f"연 변동성: {result['volatility']:.3f}")
print(f"Sharpe Ratio: {result['sharpe']:.3f}")

[*********************100%***********************]  5 of 5 completed

최적화 결과:
SPY: 0.3926
QQQ: 0.1000
GLD: 0.0640
SCHD: 0.3580
TLT: 0.0854
연 기대수익률: 0.113
연 변동성: 0.119
Sharpe Ratio: 0.805
